<center>
<h1 style="color:red; font-size:70px;"> Esercitazione 2</h1>

In [3]:
import numpy as np

<h1 style="color: blue; font-size:50px;"> Esercizio 1 - Numpy basics</h1>

NumPy (Numerical Python) è una libreria Python che consente di lavorare in modo efficiente con array, matrici e operazioni matematiche. In questo esercizio siamo interessati a prendere dimestichezza con gli array e con il concetto di deep copy. Nel python l'operazione 

```python
b = a
```

non produce una copia, ma  due oggetti coincidono (sono lo stesso array). Per separare le celle di memoria è necessario utilizzare il metodo copy, usando quindi:

```python
c = a.copy()
```

In [15]:
a = np.array([[0.5, -1], [-1, 2]], dtype=np.float32)

# Verifico caratteristiche dell'array
print("Vettore di partenza: \n")
print("Forma del contenitore: ", a.shape)
print("Rango tensore: ", a.ndim)
print("Tipo di dati contenuti: ", a.dtype)


# Utilizzo della funzione flatten, con creazione di deep copy
b = a.copy().flatten()
print("\n\nDeep copy con flattening: \n")
print("Forma del contenitore: ", b.shape)
print("Rango tensore: ", b.ndim)
print("Tipo di dati contenuti: ", b.dtype)


# Assegno zero a indici pari
print("\n\nA zero caselle pari: \n")
print("Vettore prima dell'operazione: ", b)
b[::2] = 0
print("Vettore dopo l'operazione: ", b)

Vettore di partenza: 

Forma del contenitore:  (2, 2)
Rango tensore:  2
Tipo di dati contenuti:  float32


Deep copy con flattening: 

Forma del contenitore:  (4,)
Rango tensore:  1
Tipo di dati contenuti:  float32


A zero caselle pari: 

Vettore prima dell'operazione:  [ 0.5 -1.  -1.   2. ]
Vettore dopo l'operazione:  [ 0. -1.  0.  2.]


In [8]:
a = np.array([1, 2, 3, 4], dtype=np.float32) #dtype mi consente di specificare tipo di dati

# Esempio di "memoria condivisa" (no deep copy)
# Agisco su a, ma l'effetto è evidente anche su b
b = a
print("Esempio copia:\n")

print("Primo vettore pre-operazione: ", a)
print("Secondo vettore pre-operazione: ", b)

a[1] = 1
print("\nPrimo vettore post-operazione: ", a)
print("Secondo vettore post-operazione: ", b)



# Esempio di deep copy (oggetti distinti)
# Agisco su a, ma l'effetto non è evidente su c
c = a.copy()
print("\n\n\nEsempio deep-copy:")

print("Primo vettore pre-operazione: ", a)
print("Secondo vettore pre-operazione: ", c)

a[1] = 2
print("\nPrimo vettore post-operazione: ", a)
print("Secondo vettore post-operazione: ", c)

Esempio copia:

Primo vettore pre-operazione:  [1. 2. 3. 4.]
Secondo vettore pre-operazione:  [1. 2. 3. 4.]

Primo vettore post-operazione:  [1. 1. 3. 4.]
Secondo vettore post-operazione:  [1. 1. 3. 4.]



Esempio deep-copy:
Primo vettore pre-operazione:  [1. 1. 3. 4.]
Secondo vettore pre-operazione:  [1. 1. 3. 4.]

Primo vettore post-operazione:  [1. 2. 3. 4.]
Secondo vettore post-operazione:  [1. 1. 3. 4.]
